<a href="https://colab.research.google.com/github/SaifEleslamElgalaly/food-101-classification/blob/main/bulid%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil

# 1️⃣ طلب اسم الفرع من المستخدم
branch_name = input("🚀 أدخل اسم الفرع الذي تعمل عليه: ").strip()

# 2️⃣ إعداد مسارات المستودع والبيانات
repo_url = "https://github.com/SaifEleslamElgalaly/food-101-classification.git"
repo_path = "/content/food-101-classification"
data_source = os.path.join(repo_path, "food-101-reduced")
data_copy = "/content/food-101-reduced"

# 3️⃣ استنساخ المستودع إن لم يكن موجودًا
if not os.path.exists(repo_path):
    print("📥 جاري استنساخ المستودع...")
    os.system(f"git clone {repo_url} {repo_path}")
else:
    print("✅ المستودع موجود بالفعل، سيتم التحقق من التحديثات.")

# 4️⃣ الانتقال إلى المستودع والتأكد من تحديث `main`
os.chdir(repo_path)
print("🔄 تحديث `main`...")
os.system("git checkout main")
os.system("git pull origin main")

# 5️⃣ إنشاء أو الانتقال إلى الفرع المحدد
print(f"🔀 التبديل إلى الفرع `{branch_name}`...")
os.system(f"git checkout -b {branch_name} || git checkout {branch_name}")

# 6️⃣ نسخ البيانات إلى مجلد جديد ليعمل العضو عليه دون التأثير على الأصل
if os.path.exists(data_copy):
    shutil.rmtree(data_copy)  # حذف النسخة القديمة إن وجدت

print("📂 نسخ بيانات `food-101-reduced` إلى /content...")
shutil.copytree(data_source, data_copy)

# 7️⃣ التأكيد على نجاح العملية
print(f"✅ أنت الآن تعمل على الفرع `{branch_name}`، ولديك نسخة مستقلة من البيانات في `{data_copy}`! 🚀")


🚀 أدخل اسم الفرع الذي تعمل عليه: Saif
✅ المستودع موجود بالفعل، سيتم التحقق من التحديثات.
🔄 تحديث `main`...
🔀 التبديل إلى الفرع `Saif`...
📂 نسخ بيانات `food-101-reduced` إلى /content...
✅ أنت الآن تعمل على الفرع `Saif`، ولديك نسخة مستقلة من البيانات في `/content/food-101-reduced`! 🚀


In [2]:
!cd /content/food-101-classification && git branch


* Saif
  main


In [3]:
import os

data_path = "/content/food-101-reduced"
if os.path.exists(data_path):
    print(f"✅ البيانات موجودة في: {data_path}")
    print(f"📂 عدد الملفات داخلها: {len(os.listdir(data_path))}")
else:
    print("❌ لم يتم العثور على البيانات!")


✅ البيانات موجودة في: /content/food-101-reduced
📂 عدد الملفات داخلها: 4


In [4]:
!pip install roboflow


In [5]:
!python --version

Python 3.11.12


In [6]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Sl8bWZ4B8bQFkVr4ibST")
project = rf.workspace("saif-elgalaly").project("food-101-classification")
version = project.version(1)
dataset = version.download("folder")


loading Roboflow workspace...
loading Roboflow project...


In [7]:
import os

data_path = dataset.location  # المسار اللي رجعه Roboflow
print("المجلد:", data_path)

# اعرض الملفات أو المجلدات داخله
print("محتويات المجلد:")
print(os.listdir(data_path))


المجلد: /content/food-101-classification/food-101-classification-1
محتويات المجلد:
['README.dataset.txt', 'test', 'valid', 'README.roboflow.txt', 'train']


In [8]:
import glob

train_images = glob.glob(f"{data_path}/train/*/*.jpg")
print("عدد صور التدريب:", len(train_images))


عدد صور التدريب: 10496


In [9]:
!pip install tensorflow

In [10]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras


In [11]:
!pip install keras

In [12]:
!pip show keras

Name: keras
Version: 3.8.0
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:

# نحدد المسارات
train_dir = os.path.join(data_path, 'train')
valid_dir = os.path.join(data_path, 'valid')
test_dir  = os.path.join(data_path, 'test')

# ImageDataGenerator بدون أي Augmentation
datagen = ImageDataGenerator(rescale=1./255)

# تحميل الصور من المجلدات بالحجم الجديد
train_data = datagen.flow_from_directory(train_dir, target_size=(640, 640), batch_size=32, class_mode='categorical')
valid_data = datagen.flow_from_directory(valid_dir, target_size=(640, 640), batch_size=32, class_mode='categorical')
test_data  = datagen.flow_from_directory(test_dir, target_size=(640, 640), batch_size=32, class_mode='categorical')


Found 10496 images belonging to 15 classes.
Found 2999 images belonging to 15 classes.
Found 1500 images belonging to 15 classes.


In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models

# نحدد عدد الكلاسات
num_classes = train_data.num_classes

# نبني موديل بسيط
model = models.Sequential([
    layers.Input(shape=(640, 640, 3)),  # شكل الصورة
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # عشان التصنيف متعدد
])

# نطبع ملخص الموديل
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 638, 638, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 319, 319, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 317, 317, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 158, 158, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 156, 156, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 78, 78, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 778752)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    99,680,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,775,567 (380.61 MB)

 Trainable params: 99,775,567 (380.61 MB)

 Non-trainable params: 0 (0.00 B)